## Homework_8_Solutions
## Laboratory_Workbook
### Achilles Edwin Alfred Saxby
### aes807

#### Taken extensive help from:
    - Aaron D'Souza
- The solution for this question has not been done by me
- Took entensive help and mostly all the code from Aaron and just tried to complete and learn the solution
- The whole coding aspect of this problem was difficult for me to understand and work on
    
#### Taken help from:
    - Priyanshi Singh
    - Vishwajeet Shelar
    - Akshay Penmatcha
    
#### Worked with:
    - Patrick Gitundu
    - Nonie Mathur

# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [1]:
import os
import pandas as pd
import numpy as np
import re

from sklearn import preprocessing
from sklearn.cluster import KMeans

PUIdata = os.getenv('PUIDATA')

In [2]:
fields = ['Borough', 'Community Board', 'Complaint Type', 'Descriptor', 'Incident Address', 'Incident Zip', 'Location']

In [3]:
# Load dataset
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9', usecols = fields)
data.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Complaint Type,Descriptor,Incident Zip,Incident Address,Community Board,Borough,Location
0,Street Condition,Pothole,11224,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)"
1,Opinion for the Mayor,HOUSING,NaN,NaN,0 Unspecified,Unspecified,NaN
2,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,11234,1157 EAST 57 STREET,18 BROOKLYN,BROOKLYN,"(40.62500363580505, -73.92072558378698)"
3,Illegal Parking,Blocked Hydrant,11218,722 EAST 4 STREET,12 BROOKLYN,BROOKLYN,"(40.634522428879706, -73.97479041437481)"
4,Benefit Card Replacement,Medicaid,NaN,NaN,0 Unspecified,Unspecified,NaN


# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [4]:
# Check if all Boroughs and Community Districts are represented in the Data
data['Borough'].unique()

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [5]:
data['Community Board'].unique()

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

#### How many unique values do we have? 

In [6]:
len(data['Community Board'].unique())

77

In [7]:
len(data['Borough'].unique())

6

In [8]:
len(data['Complaint Type'].unique()) 

242

In [9]:
len(data['Descriptor'].unique())

1629

In [10]:
len(data['Incident Zip'].unique())

2672

In [11]:
len(data['Incident Address'].unique())

1034112

In [12]:
len(data['Location'].unique())

949705

In [13]:
len(data)

10187766

#### Checking the representation for all the broroughs.

In [17]:
unique_borough = data.Borough.unique()
unique_community_board = data['Community Board'].unique()
print("Borough \n")
for i in data.Borough.unique():
    print(i)

Borough 

BROOKLYN
Unspecified
STATEN ISLAND
MANHATTAN
BRONX
QUEENS


In [16]:
com = data['Community Board'].unique()
com = pd.Series(com)

In [19]:
# How many unique values do we have? 
print('Total number of borough %s'%(len(unique_borough)))
print('Total number of community board %s'%(len(unique_community_board)))

Total number of borough 6
Total number of community board 77


In [20]:
# Check for duplicates? Are these plausible?
print("Total number of duplicated values %s"%data.duplicated().sum())

Total number of duplicated values 4828430


In [21]:
duplicate_data = data[data.duplicated() == True]

In [22]:
for i in unique_borough:
    print('Borough : %s \n'%i)
    print(duplicate_data[duplicate_data['Borough'] == i].count())
    print('\n')

Borough : BROOKLYN 

Complaint Type      1214927
Descriptor          1214927
Incident Zip        1116669
Incident Address     941838
Community Board     1214927
Borough             1214927
Location            1101142
dtype: int64


Borough : Unspecified 

Complaint Type      796101
Descriptor          796101
Incident Zip        519599
Incident Address    524580
Community Board     796101
Borough             796101
Location            518880
dtype: int64


Borough : STATEN ISLAND 

Complaint Type      140917
Descriptor          140917
Incident Zip        134369
Incident Address     82758
Community Board     140917
Borough             140917
Location            132153
dtype: int64


Borough : MANHATTAN 

Complaint Type      966401
Descriptor          966401
Incident Zip        839150
Incident Address    640579
Community Board     966401
Borough             966401
Location            818412
dtype: int64


Borough : BRONX 

Complaint Type      901845
Descriptor          901845
Incident Zip

In [23]:
unique_data = data[data.duplicated() == False]

In [24]:
# What about missing values? Can you detect any patterns? 
for i in unique_borough:
    print('Borough : %s \n'%i)
    print(unique_data[unique_data['Borough'] == i].count())
    print('\n')

Borough : BROOKLYN 

Complaint Type      1617005
Descriptor          1617005
Incident Zip        1593308
Incident Address    1374930
Community Board     1617005
Borough             1617005
Location            1588072
dtype: int64


Borough : Unspecified 

Complaint Type      313345
Descriptor          313345
Incident Zip        307663
Incident Address    312249
Community Board     313345
Borough             313345
Location            301970
dtype: int64


Borough : STATEN ISLAND 

Complaint Type      350081
Descriptor          350081
Incident Zip        344928
Incident Address    279499
Community Board     350081
Borough             350081
Location            343884
dtype: int64


Borough : MANHATTAN 

Complaint Type      933604
Descriptor          933604
Incident Zip        917076
Incident Address    775049
Community Board     933604
Borough             933604
Location            910312
dtype: int64


Borough : BRONX 

Complaint Type      763780
Descriptor          763780
Incident Zip

In [25]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?
unique_data.head()

,Complaint Type,Descriptor,Incident Zip,Incident Address,Community Board,Borough,Location
0,201,1134,11224,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)"
1,149,657,NaN,NaN,0 Unspecified,Unspecified,NaN
2,176,1516,11234,1157 EAST 57 STREET,18 BROOKLYN,BROOKLYN,"(40.62500363580505, -73.92072558378698)"
3,112,179,11218,722 EAST 4 STREET,12 BROOKLYN,BROOKLYN,"(40.634522428879706, -73.97479041437481)"
4,15,855,NaN,NaN,0 Unspecified,Unspecified,NaN


In [26]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
community_board = pd.Series(data['Community Board'])
community_board.head()

0      13 BROOKLYN
1    0 Unspecified
2      18 BROOKLYN
3      12 BROOKLYN
4    0 Unspecified
Name: Community Board, dtype: object

In [27]:
data['Community Index'] =community_board.str.extract('(\d+)', expand=True)
data['Community Borough'] = community_board.str.extract('([A-Z]\w{0,})', expand=True)
data.head()

,Complaint Type,Descriptor,Incident Zip,Incident Address,Community Board,Borough,Location,Community Index,Community Borough
0,201,1134,11224,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)",13,BROOKLYN
1,149,657,NaN,NaN,0 Unspecified,Unspecified,NaN,0,Unspecified
2,176,1516,11234,1157 EAST 57 STREET,18 BROOKLYN,BROOKLYN,"(40.62500363580505, -73.92072558378698)",18,BROOKLYN
3,112,179,11218,722 EAST 4 STREET,12 BROOKLYN,BROOKLYN,"(40.634522428879706, -73.97479041437481)",12,BROOKLYN
4,15,855,NaN,NaN,0 Unspecified,Unspecified,NaN,0,Unspecified


In [28]:
len(data)

10187766

In [29]:
data['Community Index'] = pd.to_numeric(data['Community Index'])
type(data['Community Index'][0])

numpy.float64

In [30]:
cd_manhattan =(data['Community Index'] > 0) & (data['Community Index'] < 13) \
& (data['Community Borough'] == 'MANHATTAN')

In [31]:
cd_brooklyn = (data['Community Index'] > 0) & (data['Community Index'] < 19) \
& (data['Community Borough'] == 'BROOKLYN')

In [32]:
cd_queens = (data['Community Index'] > 0) & (data['Community Index'] < 15) \
& (data['Community Borough'] == 'QUEENS')

In [33]:
cd_statenisland = (data['Community Index'] > 0) & (data['Community Index'] < 4) \
& (data['Community Borough'] == 'STATEN')

In [34]:
cd_bronx = (data['Community Index'] > 0) & (data['Community Index'] < 13) \
& (data['Community Borough'] == 'BRONX')

In [35]:
cd_total = cd_manhattan | cd_brooklyn | cd_queens | cd_statenisland | cd_bronx

In [36]:
unspecified = data[~cd_total]
unspecified.head()

,Complaint Type,Descriptor,Incident Zip,Incident Address,Community Board,Borough,Location,Community Index,Community Borough
1,149,657,NaN,NaN,0 Unspecified,Unspecified,NaN,0.0,Unspecified
4,15,855,NaN,NaN,0 Unspecified,Unspecified,NaN,0.0,Unspecified
6,7,912,NaN,NaN,0 Unspecified,Unspecified,NaN,0.0,Unspecified
70,149,657,NaN,NaN,0 Unspecified,Unspecified,NaN,0.0,Unspecified
72,149,1036,NaN,NaN,0 Unspecified,Unspecified,NaN,0.0,Unspecified


In [37]:
# Drop the marked districts
clean_data = data[cd_total]
clean_data.drop(['Community Index', 'Community Borough'], axis = 1, inplace=True)
clean_data.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Complaint Type,Descriptor,Incident Zip,Incident Address,Community Board,Borough,Location
0,201,1134,11224,NaN,13 BROOKLYN,BROOKLYN,"(40.57343122248129, -73.99174247588253)"
2,176,1516,11234,1157 EAST 57 STREET,18 BROOKLYN,BROOKLYN,"(40.62500363580505, -73.92072558378698)"
3,112,179,11218,722 EAST 4 STREET,12 BROOKLYN,BROOKLYN,"(40.634522428879706, -73.97479041437481)"
5,102,1135,10304,NaN,02 STATEN ISLAND,STATEN ISLAND,"(40.60687536641399, -74.0854077221027)"
7,137,829,10021,1402 2 AVENUE,08 MANHATTAN,MANHATTAN,"(40.769378795761845, -73.95793023233308)"


In [38]:
len(clean_data['Community Board'].unique())

59

In [41]:
clean_data.columns

Index([u'Complaint Type', u'Descriptor', u'Incident Zip', u'Incident Address',
       u'Community Board', u'Borough', u'Location'],
      dtype='object')

In [42]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
complaints = clean_data.groupby('Community Board').count()
complaints.head()

,Complaint Type,Descriptor,Incident Zip,Incident Address,Borough,Location
Community Board,,,,,,
01 BRONX,74631,74631,74435,64772,74631,73748
01 BROOKLYN,185057,185057,184497,144518,185057,182790
01 MANHATTAN,77974,77974,77877,49134,77974,77186
01 QUEENS,171484,171484,170826,134409,171484,169154
01 STATEN ISLAND,182713,182713,182043,144748,182713,180800


In [43]:
complaints.drop([u'Descriptor', u'Incident Zip', u'Borough', u'Location', u'Incident Address'], axis = 1, inplace = True)
complaints.head()

,Complaint Type
Community Board,
01 BRONX,74631
01 BROOKLYN,185057
01 MANHATTAN,77974
01 QUEENS,171484
01 STATEN ISLAND,182713


In [44]:
complaints.reset_index(inplace=True)
complaints.rename(columns={'Complaint Type' : 'Complaint Count'}, inplace=True)
complaints.head()

,Community Board,Complaint Count
0,01 BRONX,74631
1,01 BROOKLYN,185057
2,01 MANHATTAN,77974
3,01 QUEENS,171484
4,01 STATEN ISLAND,182713


In [45]:
complaints['Rank'] = complaints['Community Board'].rank(ascending=False)
complaints.head()

,Community Board,Complaint Count,Rank
0,01 BRONX,74631,59.0
1,01 BROOKLYN,185057,58.0
2,01 MANHATTAN,77974,57.0
3,01 QUEENS,171484,56.0
4,01 STATEN ISLAND,182713,55.0


In [46]:
complaints.sort(columns='Rank').head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Community Board,Complaint Count,Rank
58,18 BROOKLYN,156100,1.0
57,17 BROOKLYN,191720,2.0
56,16 BROOKLYN,85467,3.0
55,15 BROOKLYN,146129,4.0
54,14 QUEENS,103396,5.0


In [47]:
# km = KMeans(random_state = 123, n_clusters = len(data.index))
# res = km.fit(data)
# print (res.labels_.max())

In [48]:
# Safe reduced data frame (Community District level)

# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

#### CSV directly downloaded
#### Uploaded via Jupyter Notebooks into PUIdata folder

In [63]:
# Population by Community District
df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")
df_pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [64]:
# Check variables in file
print("There are %d variables in the csv file"%len(df_pop.columns))

There are 158 variables in the csv file


In [65]:
# How many community districts are in file? 
print("There are %d community districts in the file"%len(df_pop.cd_id.unique()))

There are 59 community districts in the file


In [66]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311count
df_pop.columns

Index([u'FIPS', u'cd_id', u'Total Population',
       u'Population Density (per sq. mile)', u'% Total Population: Male',
       u'% Total Population: 18 to 24 Years',
       u'% Total Population: 25 to 34 Years',
       u'% Total Population: 35 to 44 Years',
       u'% Population 5 Years And Over: Speak Only English',
       u'% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       u'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       u'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       u'% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       u'% Employed Civilian Population 16 Years And Over: Service occupations',
       u'% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       

In [67]:
# Save data frame
demographics = df_pop[['FIPS','cd_id','Median household income (In 2014 Inflation Adjusted Dollars)']]
demographics.head()

,FIPS,cd_id,Median household income (In 2014 Inflation Adjusted Dollars)
0,3603701,BX08,54224
1,3603702,BX12,44906
2,3603703,BX10,54962
3,3603704,BX11,47910
4,3603705,BX03,22041


In [69]:
demographics['cd_id'][15] = 'MN11'
demographics.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,FIPS,cd_id,Median household income (In 2014 Inflation Adjusted Dollars)
0,3603701,BX08,54224
1,3603702,BX12,44906
2,3603703,BX10,54962
3,3603704,BX11,47910
4,3603705,BX03,22041


#### CSV directly downloaded
#### Uploaded via Jupyter Notebooks into PUIdata folder

In [71]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [72]:
df_infr.columns

Index([u'FIPS', u'Geographic Identifier', u'Qualifying Name', u'Households',
       u'Households: With An Internet Subscription',
       u'Households: Dial-Up Alone', u'Households: Dsl',
       u'Households: With Mobile Broadband',
       u'Households: Without Mobile Broadband', u'Households: Cable Modem',
       u'Households: With Mobile Broadband.1',
       u'Households: Without Mobile Broadband.1', u'Households: Fiber-Optic',
       u'Households: With Mobile Broadband.2',
       u'Households: Without Mobile Broadband.2',
       u'Households: Satellite Internet Service',
       u'Households: With Mobile Broadband.3',
       u'Households: Without Mobile Broadband.3',
       u'Households: Two or More Fixed Broadband Types, or Other',
       u'Households: With Mobile Broadband.4',
       u'Households: Without Mobile Broadband.4',
       u'Households: Mobile Broadband Alone or With Dialup',
       u'Households: Internet Access Without A Subscription',
       u'Households: No Internet Acc

In [73]:
# Check variables in file
print("There are %d variables in the csv file"%len(df_infr.columns))

There are 31 variables in the csv file


In [74]:
# How many community districts are in file? 
print("There are %d community districts in the file"%len(df_infr['Qualifying Name'].unique()))

There are 55 community districts in the file


In [75]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
internet = pd.DataFrame(df_infr['Qualifying Name'])
internet['Total Mobile Subscription'] = df_infr['Households: With Mobile Broadband'] + \
df_infr['Households: With Mobile Broadband.1'] + df_infr['Households: With Mobile Broadband.2'] + \
df_infr['Households: With Mobile Broadband.3'] + df_infr['Households: With Mobile Broadband.4'] 
internet.head()

,Qualifying Name,Total Mobile Subscription
0,"NYC-Bronx Community District 8--Riverdale, New...",15359
1,"NYC-Bronx Community District 12--Wakefield, Ne...",10486
2,"NYC-Bronx Community District 10--Co-op City, N...",11425
3,NYC-Bronx Community District 11--Pelham Parkwa...,10748
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",14907


In [76]:
# Aggregate internet type by high and low connections
internet.sort(columns='Total Mobile Subscription', ascending=False).head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,Qualifying Name,Total Mobile Subscription
16,NYC-Manhattan Community District 8--Upper East...,60021
17,NYC-Manhattan Community District 7--Upper West...,45095
23,NYC-Manhattan Community District 1 & 2--Batter...,39655
22,NYC-Manhattan Community District 1 & 2--Batter...,39655
20,NYC-Manhattan Community District 6--Murray Hil...,38116


In [ ]:
# Save data frame 

# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [77]:
# Harmonize identifier of dataframe 1
community_board = pd.Series(complaints['Community Board'])
complaints['Community Index'] =community_board.str.extract('(\d+)', expand=True)
complaints['Community Borough'] = community_board.str.extract('([A-Z]\w{0,})', expand=True)
complaints.head()

,Community Board,Complaint Count,Rank,Community Index,Community Borough
0,01 BRONX,74631,59.0,01,BRONX
1,01 BROOKLYN,185057,58.0,01,BROOKLYN
2,01 MANHATTAN,77974,57.0,01,MANHATTAN
3,01 QUEENS,171484,56.0,01,QUEENS
4,01 STATEN ISLAND,182713,55.0,01,STATEN


In [78]:
complaints['Abbr'] = complaints['Community Borough']
complaints['Abbr'] = complaints['Abbr'].str.replace(pat='BROOKLYN', repl='BK')
complaints['Abbr'] = complaints['Abbr'].str.replace(pat='MANHATTAN', repl='MN')
complaints['Abbr'] = complaints['Abbr'].str.replace(pat='STATEN', repl='SI')
complaints['Abbr'] = complaints['Abbr'].str.replace(pat='BRONX', repl='BX')
complaints['Abbr'] = complaints['Abbr'].str.replace(pat='QUEENS', repl='QN')
complaints['cd_id'] = complaints[['Abbr', 'Community Index']].apply(lambda x: ''.join(x), axis = 1)
complaints.drop(['Community Index', 'Abbr'], axis=1, inplace=True)
complaints.head()

,Community Board,Complaint Count,Rank,Community Borough,cd_id
0,01 BRONX,74631,59.0,BRONX,BX01
1,01 BROOKLYN,185057,58.0,BROOKLYN,BK01
2,01 MANHATTAN,77974,57.0,MANHATTAN,MN01
3,01 QUEENS,171484,56.0,QUEENS,QN01
4,01 STATEN ISLAND,182713,55.0,STATEN,SI01


In [79]:
# Harmonize identifier of dataframe 2
demographics.head()

,FIPS,cd_id,Median household income (In 2014 Inflation Adjusted Dollars)
0,3603701,BX08,54224
1,3603702,BX12,44906
2,3603703,BX10,54962
3,3603704,BX11,47910
4,3603705,BX03,22041


In [80]:
# Harmonize identifier of dataframe 3
internet.head()

,Qualifying Name,Total Mobile Subscription
0,"NYC-Bronx Community District 8--Riverdale, New...",15359
1,"NYC-Bronx Community District 12--Wakefield, Ne...",10486
2,"NYC-Bronx Community District 10--Co-op City, N...",11425
3,NYC-Bronx Community District 11--Pelham Parkwa...,10748
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",14907


In [82]:
community_district = internet['Qualifying Name'].str.split(' ')
community_district = pd.Series(community_district)

for i in range(len(community_district)):
    internet.loc[i,'Borough'] = str(community_district[i][0])[4:]

internet.head()

,Qualifying Name,Total Mobile Subscription,Borough
0,"NYC-Bronx Community District 8--Riverdale, New...",15359,Bronx
1,"NYC-Bronx Community District 12--Wakefield, Ne...",10486,Bronx
2,"NYC-Bronx Community District 10--Co-op City, N...",11425,Bronx
3,NYC-Bronx Community District 11--Pelham Parkwa...,10748,Bronx
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",14907,Bronx


In [83]:
internet['Abbr'] = internet['Borough']
internet['Abbr'] = internet['Abbr'].str.replace(pat='Brooklyn', repl='BK')
internet['Abbr'] = internet['Abbr'].str.replace(pat='Manhattan', repl='MN')
internet['Abbr'] = internet['Abbr'].str.replace(pat='Staten', repl='SI')
internet['Abbr'] = internet['Abbr'].str.replace(pat='Bronx', repl='BX')
internet['Abbr'] = internet['Abbr'].str.replace(pat='Queens', repl='QN')
internet.head()

,Qualifying Name,Total Mobile Subscription,Borough,Abbr
0,"NYC-Bronx Community District 8--Riverdale, New...",15359,Bronx,BX
1,"NYC-Bronx Community District 12--Wakefield, Ne...",10486,Bronx,BX
2,"NYC-Bronx Community District 10--Co-op City, N...",11425,Bronx,BX
3,NYC-Bronx Community District 11--Pelham Parkwa...,10748,Bronx,BX
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",14907,Bronx,BX


In [85]:
internet['Community Index 1'] = internet['Qualifying Name'].str.extract('(\d+)', expand=True)
internet['Community Index 2'] = internet['Qualifying Name'].str.extract('(\d+\W+\d+)', expand=True)
internet['Community Index 3'] = internet['Qualifying Name'].str.extract('(\d+)(?!.*\d)', expand = True)

internet['Community Index 2'] = internet['Community Index 2'].fillna(0)
community_index = pd.Series(internet['Community Index 2'])

internet.head()

,Qualifying Name,Total Mobile Subscription,Borough,Abbr,Community Index 1,Community Index 2,Community Index 3
0,"NYC-Bronx Community District 8--Riverdale, New...",15359,Bronx,BX,8,0,8
1,"NYC-Bronx Community District 12--Wakefield, Ne...",10486,Bronx,BX,12,0,12
2,"NYC-Bronx Community District 10--Co-op City, N...",11425,Bronx,BX,10,0,10
3,NYC-Bronx Community District 11--Pelham Parkwa...,10748,Bronx,BX,11,0,11
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",14907,Bronx,BX,3,3 & 6,6


In [86]:
flag = 0
for i in range(len(internet['Community Index 2'])):
    if (flag == 1):
        internet['Community Index 1'][i] = internet['Community Index 3'][i]
        flag = 0
    if (internet['Community Index 2'][i] != 0):
        flag = 1
        
internet.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Qualifying Name,Total Mobile Subscription,Borough,Abbr,Community Index 1,Community Index 2,Community Index 3
0,"NYC-Bronx Community District 8--Riverdale, New...",15359,Bronx,BX,8,0,8
1,"NYC-Bronx Community District 12--Wakefield, Ne...",10486,Bronx,BX,12,0,12
2,"NYC-Bronx Community District 10--Co-op City, N...",11425,Bronx,BX,10,0,10
3,NYC-Bronx Community District 11--Pelham Parkwa...,10748,Bronx,BX,11,0,11
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",14907,Bronx,BX,3,3 & 6,6


In [87]:
internet['cd_id'] = internet[['Abbr', 'Community Index 1']].apply(lambda x: ''.join(x), axis = 1)
internet.head()

,Qualifying Name,Total Mobile Subscription,Borough,Abbr,Community Index 1,Community Index 2,Community Index 3,cd_id
0,"NYC-Bronx Community District 8--Riverdale, New...",15359,Bronx,BX,8,0,8,BX8
1,"NYC-Bronx Community District 12--Wakefield, Ne...",10486,Bronx,BX,12,0,12,BX12
2,"NYC-Bronx Community District 10--Co-op City, N...",11425,Bronx,BX,10,0,10,BX10
3,NYC-Bronx Community District 11--Pelham Parkwa...,10748,Bronx,BX,11,0,11,BX11
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",14907,Bronx,BX,3,3 & 6,6,BX3


In [88]:
internet.drop(['Qualifying Name', 'Borough', 'Community Index 1', 'Community Index 2', 'Community Index 3', 'Abbr'],\
              axis = 1, inplace=True)
internet.head()

,Total Mobile Subscription,cd_id
0,15359,BX8
1,10486,BX12
2,11425,BX10
3,10748,BX11
4,14907,BX3


In [99]:
# Link the 3 dataframes
complaints_demographics = pd.merge(complaints, demographics, on='cd_id', how = 'outer')
complaints_demographics_internet = pd.merge(complaints_demographics, internet, on='cd_id', how = 'outer')
len(complaints_demographics_internet)

98

In [102]:
complaints_demographics_internet.columns

Index([u'Community Board', u'Complaint Count', u'Rank', u'Community Borough',
       u'cd_id', u'FIPS',
       u'Median household income (In 2014 Inflation Adjusted Dollars)',
       u'Total Mobile Subscription'],
      dtype='object')

In [103]:
complaints_demographics_internet.drop(['Community Board', 'Community Borough'], axis = 1, inplace=True)
complaints_demographics_internet.head()

,Complaint Count,Rank,cd_id,FIPS,Median household income (In 2014 Inflation Adjusted Dollars),Total Mobile Subscription
0,74631.0,59.0,BX01,3603710.0,21116.0,NaN
1,185057.0,58.0,BK01,3604001.0,57818.0,NaN
2,77974.0,57.0,MN01,3603810.0,120190.0,NaN
3,171484.0,56.0,QN01,3604101.0,56269.0,NaN
4,182713.0,55.0,SI01,3603903.0,57602.0,NaN


In [ ]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?